In [44]:
import os
from configparser import ConfigParser
from typing import List
import pandas as pd
import time
import pymysql
import logging


class TiDBHypo:
    def __init__(self, db_name):
        config_raw = ConfigParser()
        config_raw.read('configure.ini')
        defaults = config_raw.defaults()
        if db_name is None:
            db_name = 'test'
        self.host = defaults.get('tidb_ip')
        self.port = defaults.get('tidb_port')
        self.user = defaults.get('tidb_user')
        self.password = defaults.get('tidb_password')
        self.database = defaults.get('tidb_database')
        self.db_name = db_name
        self.conn = None
        self.create_connection()
        self._cursor = self.conn.cursor()
        

    def close(self):
        self.conn.close()


    def create_connection(self):
        if self.conn:
            self.close()
        self.conn = pymysql.connect(host='127.0.0.1',
                     port=4000,
                     user='root',
                     password='',
                     database="{}".format(self.db_name),
                     local_infile=True)


    def exec_fetch(self, statement, one=True):
        self._cursor.execute(statement)
        if one:
            print(self._cursor.fetchone())
            return self._cursor.fetchone()
        return self._cursor.fetchall()


    def execute_create_hypo(self, index):
        schema = index.split("#")
        table_name = schema[0]
        idx_cols = schema[1]
        idx_name = f"hypo_{table_name}_{idx_cols}_idx" 
        statement = (
            f"create index {idx_name} type hypo "
            f"on {table_name} ({idx_cols})"
        )
        print(statement)
        cur = self.conn.cursor()
        cur.execute("create index hypo_aka_name_name_pcode_cf_idx type hypo on aka_name (name_pcode_cf)")
        return cur.fetchall()


    def get_hypo_indexes_from_one_table(self, table_name):
        statement = f"show create table {table_name}"
        result = self.exec_fetch(statement)
        hypo_indexes = []
        for line in result[1].split("\n"):
            if "HYPO INDEX" in line:
                tmp = line.split("`")
                hypo_indexes.append(tmp[1])
        return hypo_indexes
    
    
    # TODO
    def execute_delete_hypo(self, oid):
        sql = "select * from hypopg_drop_index(" + str(oid) + ");"
        cur = self.conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        flag = str(rows[0][0])
        if flag == "t":
            return True
        return False

    def _cleanup_query(self, query):
        for query_statement in query.text.split(";"):
            if "drop view" in query_statement:
                self.exec_only(query_statement)
                
    def _prepare_query(self, query):
        for query_statement in query.text.split(";"):
            if "create view" in query_statement:
                try:
                    self.exec_only(query_statement)
                except Exception as e:
                    logging.error(e)
            elif "select" in query_statement or "SELECT" in query_statement:
                return query_statement
    
    def _get_plan(self, query):
        query_text = self._prepare_query(query)
        statement = f"explain format='verbose' {query_text}"
        query_plan = self.exec_fetch(statement, False)
        for line in query_plan:
            if "stats:pseudo" in line[5]:
                raise Exception("plan with pseudo stats " + str(query_plan))
        self._cleanup_query(query)
        return query_plan

    def get_queries_cost(self, query_list):
        cost_list: List[float] = list()
        for i, query in enumerate(query_list):
            query_plan = self._get_plan(query)
            cost = query_plan[0][2]
            cost_list.append(float(cost))
        return cost_list

    def get_storage_cost(self, oid_list):
        costs = list()
        cur = self.conn.cursor()
        for i, oid in enumerate(oid_list):
            if oid == 0:
                continue
            sql = "select * from hypopg_relation_size(" + str(oid) +");"
            cur.execute(sql)
            rows = cur.fetchall()
            df = pd.DataFrame(rows)
            cost_info = str(df[0][0])
            cost_long = int(cost_info)
            costs.append(cost_long)
        return costs

    def execute_sql(self, sql):
        cur = self.conn.cursor()
        cur.execute(sql)
        self.conn.commit()

    def get_tables(self, schema):
        tables_sql = 'select tablename from pg_tables where schemaname=\''+schema+'\';'
        cur = self.conn.cursor()
        cur.execute(tables_sql)
        rows = cur.fetchall()
        table_names = list()
        for i, table_name in enumerate(rows):
            table_names.append(table_name[0])
        return table_names

    def get_attributes(self, table_name, schema):
        attrs_sql = 'select column_name, data_type from information_schema.columns where table_schema=\''+schema+'\' and table_name=\''+table_name+'\''
        cur = self.conn.cursor()
        cur.execute(attrs_sql)
        rows = cur.fetchall()
        attrs = list()
        for i, attr in enumerate(rows):
            info = str(attr[0]) + "#" + str(attr[1])
            attrs.append(info)
        return attrs
    
    
    def test(self):
        sql = "select * from aka_name limit 100;"
        cur = self.conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        print(rows)

In [48]:
a = TiDBHypo("imdbload")
# b = a.execute_create_hypo("aka_name#surname_pcode")
b = a.get_hypo_indexes_from_one_table("aka_name")
b
# a.test()


('aka_name', 'CREATE TABLE `aka_name` (\n  `id` int(11) NOT NULL,\n  `person_id` int(11) NOT NULL,\n  `name` varchar(512) DEFAULT NULL,\n  `imdb_index` varchar(3) DEFAULT NULL,\n  `name_pcode_cf` varchar(11) DEFAULT NULL,\n  `name_pcode_nf` varchar(11) DEFAULT NULL,\n  `surname_pcode` varchar(11) DEFAULT NULL,\n  `md5sum` varchar(65) DEFAULT NULL,\n  PRIMARY KEY (`id`) /*T![clustered_index] CLUSTERED */\n) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin')


TypeError: 'NoneType' object is not subscriptable

In [59]:

string = "KEY `hypo_aka_name_name_pcode_nf_idx` (`name_pcode_nf`) /* HYPO INDEX */"

if "HYPO INDEX" in string:
    tmp = string.split("`")
    idx_name = tmp[1]
    table_name = "aka_name"
    hypo = f"{table_name}.{idx_name}"
    print(hypo)
    
    

aka_name.hypo_aka_name_name_pcode_nf_idx


In [62]:
import numpy as np

np.zeros(60, dtype=np.cfloat)

array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

In [58]:
import pymysql


def get_hypo_indexes_from_table(conn, table_name):
    statement = f"show create table {table_name}"
    cur = conn.cursor()
    result = cur.execute(statement)
    hypo_indexes = []
    for line in result[1].split("\n"):
        if "HYPO INDEX" in line:
            tmp = line.split("`")
            hypo_indexes.append(tmp[1])
    return hypo_indexes
    
    

conn = pymysql.connect(host='127.0.0.1',
                     port=4000,
                     user='root',
                     password='',
                     database="imdbload",
                     local_infile=True)

cur = conn.cursor()



statement = "create index hypo_aka_name_name_pcode_nf_idx type hypo on aka_name (name_pcode_nf);"
s2 = "show create table aka_name;"


cur.execute(statement)

print(get_hypo_indexes_from_table(conn, "aka_name"))

cur.execute(s2)

print(cur.fetchall())

TypeError: 'int' object is not subscriptable